# Preprocessing and Exploratory Data Analysis

### Import libraries and read source FASTA file

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import uuid

fpath_fasta = '../data/current_Bacteria_unaligned.fa'
seq_data = open(fpath_fasta, 'r').read()
lines = seq_data.split('\n')

### Function to process sample rows

In [2]:
def get_seq_dict(desc, seq):
    try:
        keys = ['domain', 'phylum', 'class', 'subclass', 'order', 'suborder', 'family', 'genus']
        txn_info = desc.split('rootrank;')[1].split(';')
        seq_info = {}
        for i, key in enumerate(keys):
            try:
                idx = txn_info.index(key)
                seq_info[key] = txn_info[idx-1].replace('"', '')
            except ValueError:
                seq_info[key] = None
        seq_info['id'] = uuid.uuid4().hex[:10].upper()
        seq_info['seq_len'] = len(seq)
        seq_info['sequence'] = seq.upper()
    except Exception as e:
        print(i)
        print(txn_info)
        print(desc)
        raise IndexError(e)
    
    return seq_info

### Compile DNA sequence samples as an array of dicts

In [3]:
desc = ''
seq = ''

seq_dicts = []

for i, line in enumerate(tqdm(lines, total=len(lines))):
    if line.startswith('>'):
        if len(desc) > 0 and len(seq) > 0:
            seq_dicts.append(get_seq_dict(desc, seq))
        desc = line
        seq = ''
    else:
        seq += line

### Create dataframe from sequence dicts

In [4]:
df_seqs = pd.DataFrame(seq_dicts)

### Filter by sequence length

In [111]:
df_len_filtered = df_seqs[df_seqs['seq_len'] >= 400]
df_len_filtered = df_len_filtered[df_seqs['seq_len'] <= 440]

df_len_filtered = df_len_filtered[df_len_filtered.apply(lambda r: set(list(r['sequence'])) == {'G', 'C', 'A', 'T'}, axis=1, reduce=True)]
df_len_filtered = df_len_filtered.drop_duplicates(subset='sequence')

print(df_len_filtered.size)


/home/srivathsa/miniconda3/envs/superpacs/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: The reduce argument is deprecated and will be removed in a future version. You can specify result_type='reduce' to try to reduce the result to the original dimensions
  after removing the cwd from sys.path.


353947


### Filter by sample counts per phylum

In [112]:
top_phylum_names = list(df_len_filtered['phylum'].value_counts()[:3].index)
print(top_phylum_names)

df_phy_filtered = df_len_filtered[df_len_filtered['phylum'].isin(top_phylum_names)]
print(df_phy_filtered.shape)

['Proteobacteria', 'Firmicutes', 'Actinobacteria']
(22947, 11)


### Filter by sample counts per class

In [113]:
top_class_names = list(df_phy_filtered['class'].value_counts()[:5].index)
print(top_class_names)

df_class_filtered = df_phy_filtered[df_phy_filtered['class'].isin(top_class_names)]
print(df_class_filtered.shape)

['Alphaproteobacteria', 'Actinobacteria', 'Gammaproteobacteria', 'Clostridia', 'Betaproteobacteria']
(18406, 11)


### Filter by sample counts per order

In [114]:
top_order_names = list(df_class_filtered['order'].value_counts()[:19].index)
print(top_order_names)

df_order_filtered = df_class_filtered[df_class_filtered['order'].isin(top_order_names)]
print(df_order_filtered.shape)

['Clostridiales', 'Actinomycetales', 'Burkholderiales', 'Rhizobiales', 'Pseudomonadales', 'Enterobacteriales', 'Sphingomonadales', 'Rhodobacterales', 'Bifidobacteriales', 'Nitrosomonadales', 'Coriobacteriales', 'Rhodospirillales', 'Xanthomonadales', 'Rhodocyclales', 'Alteromonadales', 'Caulobacterales', 'Acidimicrobiales', 'Oceanospirillales', 'Vibrionales']
(14585, 11)


### Filter by sample counts per family

In [115]:
top_family_names = list(df_order_filtered['family'].value_counts()[:65].index)

df_family_filtered = df_order_filtered[df_order_filtered['family'].isin(top_family_names)]
print(df_family_filtered.shape)

(12619, 11)


### Filter by sample counts per genus

In [116]:
top_genus_names = list(df_family_filtered['genus'].value_counts()[:393].index)

df_final = df_family_filtered[df_family_filtered['genus'].isin(top_genus_names)]
print(df_final.shape)

(9174, 11)


### Sample count per phylum

In [117]:
df_final['phylum'].value_counts()

Proteobacteria    5083
Actinobacteria    2211
Firmicutes        1880
Name: phylum, dtype: int64

### Sample count per class

In [118]:
df_final['class'].value_counts()

Actinobacteria         2211
Clostridia             1880
Betaproteobacteria     1746
Gammaproteobacteria    1720
Alphaproteobacteria    1617
Name: class, dtype: int64

### Sample count per order

In [119]:
df_final['order'].value_counts()

Clostridiales        1880
Actinomycetales      1351
Burkholderiales      1095
Pseudomonadales       788
Rhizobiales           762
Bifidobacteriales     493
Nitrosomonadales      413
Sphingomonadales      362
Enterobacteriales     324
Coriobacteriales      318
Rhodocyclales         238
Xanthomonadales       226
Alteromonadales       205
Rhodobacterales       195
Caulobacterales       174
Rhodospirillales      124
Vibrionales           106
Oceanospirillales      71
Acidimicrobiales       49
Name: order, dtype: int64

### Sample count per family

In [120]:
df_final['family'].value_counts()

Lachnospiraceae                      752
Ruminococcaceae                      625
Pseudomonadaceae                     576
Bifidobacteriaceae                   493
Burkholderiaceae                     446
Nitrosomonadaceae                    413
Comamonadaceae                       350
Enterobacteriaceae                   324
Sphingomonadaceae                    323
Coriobacteriaceae                    318
Rhodocyclaceae                       238
Micrococcaceae                       238
Mycobacteriaceae                     221
Bradyrhizobiaceae                    212
Moraxellaceae                        212
Rhodobacteraceae                     195
Xanthomonadaceae                     174
Microbacteriaceae                    173
Peptostreptococcaceae                170
Caulobacteraceae                     167
Rhizobiaceae                         162
Propionibacteriaceae                 161
Oxalobacteraceae                     141
Clostridiaceae 1                     119
Burkholderiales_

### Sample count per genus

In [121]:
df_final['genus'].value_counts()[:15]

Pseudomonas          525
Bifidobacterium      484
Faecalibacterium     458
Mycobacterium        221
Nitrosomonas         207
Nitrosospira         206
Blautia              197
Polynucleobacter     191
Sphingomonas         169
Bradyrhizobium       168
Propionibacterium    137
Collinsella          135
Burkholderia         133
Rhizobium            129
Arthrobacter         126
Name: genus, dtype: int64

### Drop extra columns and create final dataframe

In [122]:
df_final = df_final.reset_index(drop=True)
df_final = df_final.drop(['seq_len', 'subclass', 'suborder', 'domain'], axis=1)

### Shuffle and display the first 10 rows of the dataset

In [123]:
df_final.sample(frac=1)[:10]

,class,family,genus,id,order,phylum,sequence
417,Actinobacteria,Mycobacteriaceae,Mycobacterium,0BA04EBBA1,Actinomycetales,Actinobacteria,ACGTGGGTAATCTGCCCTGCACTTTGGGATAAGCCTGGGAAACTGG...
7805,Clostridia,Lachnospiraceae,Catonella,51C76D1F32,Clostridiales,Firmicutes,GATGAACGCAGGCGGCGTGCCTAACACATGCAAGTCGAACGGAGAT...
8388,Clostridia,Peptostreptococcaceae,Peptostreptococcus,18C889B711,Clostridiales,Firmicutes,GATGAACGCTGGCGGCGTGCCTAACACATGCAAGTCGAGCGAGGGT...
707,Actinobacteria,Microbacteriaceae,Microbacterium,F5A5E83098,Actinomycetales,Actinobacteria,GTCGAACGGTGAAGCCCAGCTTGCTGGGTGGATCAGTGGCGAACGG...
5644,Gammaproteobacteria,Alteromonadaceae,Marinobacter,748E5034DA,Alteromonadales,Proteobacteria,TCAGGCCTAACACATGCAAGTCGAGCGGTAACAGGGGGTGCTTGCA...
557,Actinobacteria,Geodermatophilaceae,Modestobacter,487AD84FCD,Actinomycetales,Actinobacteria,ATGGCTCAGGACGAACGCTGGCGGCGTGCTTAACACATGCAAGTCG...
1135,Actinobacteria,Propionibacteriaceae,Propionibacterium,03DDC3C596,Actinomycetales,Actinobacteria,AGTCGAACGGAAAGGCCCTGCTTTTGTGGGGTGCTCGAGTGGCGAA...
5073,Betaproteobacteria,Nitrosomonadaceae,Nitrosomonas,8FF4BD8983,Nitrosomonadales,Proteobacteria,TGCCAGCAGCCGCGGTAATACGTAGGGTGCGAGCGTTAATCGGAAT...
8316,Clostridia,Lachnospiraceae,Fusicatenibacter,095948F478,Clostridiales,Firmicutes,TTATCCGGATTTACTGGGTGTAAAGGGAGCGTAGACGGCAAGGCAA...
3058,Alphaproteobacteria,Rhizobiaceae,Rhizobium,D1BCA1F7A4,Rhizobiales,Proteobacteria,GCAAGTCGAACGCCCCGCAAGGGGAGTGGCAGACGGGTGAGTAACG...


### Save the final processed dataframe as CSV

In [124]:
df_final.to_csv('../data/taxa_400.csv')